<a href="https://colab.research.google.com/github/mdjamina/model_relatives_psr_fr/blob/model_v2/src/psr_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation modules

In [1]:
!pip install datasets
!pip install -U transformers[torch]
!pip install -U accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.7 MB/s et

# Chargement du dataset

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from datasets import load_dataset

#chargement de dataset à partir de Hugging Face
raw_datasets = load_dataset("djamina/relatives_psr")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/392 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/99 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/55 [00:00<?, ? examples/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
        num_rows: 392
    })
    validation: Dataset({
        features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
        num_rows: 99
    })
    test: Dataset({
        features: ['id', 'tokens', 'psr_tags', 'psr_seq_tags'],
        num_rows: 55
    })
})

# Préparation des données

In [5]:
task = "psr" # Should be one of "psr" or "psr_seq"

model_checkpoint = "almanach/camembert-large"
batch_size = 16

## Tokenizer




In [8]:
 #Importation de la classe AutoTokenizer de la bibliothèque 'transformers'
from transformers import CamembertTokenizerFast


tokenizer = CamembertTokenizerFast.from_pretrained(model_checkpoint, force_download=True)

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

In [9]:
# Vérification si le tokenizer est une version rapide

tokenizer.is_fast

True

### Traitement des labels

In [ ]:
# Explication de la sortie des tokens :
# - '<s>' : Token de début de séquence (utilisé pour indiquer le début d'une phrase).
# - Les caractères '▁' indiquent le début d'un nouveau mot ou d'un sous-mot, ceci est dû à l'utilisation de la tokenisation par sous-mots (subword tokenization) utilisée par le modèle CamemBERT.
# - Les sous-mots permettent de gérer efficacement les mots rares ou inconnus en les décomposant en unités plus petites.
# - Par exemple, '▁Mes' est le mot "Mes", '▁collègues' est le mot "collègues", mais 's', 'cell', 'ons' sont des sous-mots formant le mot "scellons".
# - Cette méthode améliore la capacité du modèle à généraliser à partir de morceaux de mots, surtout en présence de vocabulaire diversifié.


In [10]:
def show(words, labels,label_names):
  line1 = ""
  line2 = ""
  for word, label in zip(words, labels):
      full_label = label_names[label]
      max_length = max(len(word), len(full_label))
      line1 += word + " " * (max_length - len(word) + 1)
      line2 += full_label + " " * (max_length - len(full_label) + 1)

  # Affichage aligné des tokens et de leurs tags
  print(line1)
  print(line2)

In [11]:

# Extraction des features 'psr_tags' à partir du dataset
label_names = raw_datasets["train"].features[f"{task}_tags"].feature.names
print(f'label_names : {label_names}')
print("-----------------")

words = raw_datasets["train"][15]["tokens"]
labels = raw_datasets["train"][15][f"{task}_tags"]

show(words, labels,label_names)



label_names : ['O', 'DET', 'APPO', 'AMBIGUE']
-----------------
Notre peuple n' est pas formé d' un peu plus de soixante millions d' individus qui cohabiteraient . Il est indivisible , précisément parce que ce qui le tient est plus fort que de simples règles ou des organisations . C' est un engagement chaque jour répété qui fait que notre citoyenneté n' est jamais abstraite et froide , mais qu ' elle est pleine et entière par ce lien fraternel qui nous unit et dont nous devons retrouver la vigueur . 
O     O      O  O   O   O     O  O  O   O    O  O        O        O  O         DET O              O O  O   O           O O           O     O   O  DET O  O     O   O    O    O   O  O       O      O  O   O             O O  O   O  O          O      O    O      O   O    O   O     O           O  O   O      O         O  O      O O    O  O O    O   O      O  O       O   O  O    O         DET O    O    O  DET  O    O      O         O  O       O 


In [12]:
# cette méthode permet d'aligner les labels avec les nouveaux tokens générés par le tokenizer

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [13]:
# cette méthode permet de tokeniser le dataset (par ligne) et d'aligner les labels avec les tokens générés par le tokenizer

def tokenize_and_align_labels(rows):
    tokenized_inputs = tokenizer(
        rows["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = rows[f"{task}_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs




### Tokenization

Application de la méthode 'tokenize_and_align_labels' sur le dataset 'raw_datasets'


pour obtenir le dataset 'tokenized_datasets' qui sera utilisé pour l'entraînement du modèle

In [14]:
# psr_tags

tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/392 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/55 [00:00<?, ? examples/s]

In [15]:
# psr_tags_seq
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Map:   0%|          | 0/392 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 281
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 71
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 88
    })
})

In [ ]:
print(tokenized_datasets["train"][15]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


# Evaluate

In [16]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Méthode pour calculer les métriques d'évaluation
# On utilise les métriques de classification NER (Named Entity Recognition)
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [
        [label_names[l] for l in label if l != -100] for label in labels
    ]
    true_predictions = [
        [label_names[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Flatten the lists to calculate metrics
    true_labels_flat = [item for sublist in true_labels for item in sublist]
    true_predictions_flat = [item for sublist in true_predictions for item in sublist]

    results = {
        "precision": precision_score(true_labels_flat, true_predictions_flat, average='macro',zero_division=1),
        "recall": recall_score(true_labels_flat, true_predictions_flat, average='macro'),
        "f1": f1_score(true_labels_flat, true_predictions_flat, average='macro'),
        "accuracy": accuracy_score(true_labels_flat, true_predictions_flat),
    }
    return results

# Train

In [17]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [18]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    2,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    1,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0, -100]])

In [21]:
label_names = raw_datasets["train"].features[f"{task}_tags"].feature.names

id2label = {x:y for x,y in enumerate(label_names)}
label2id = {y:x for x,y in enumerate(label_names)}
label2id

{'O': 0, 'DET': 1, 'APPO': 2, 'AMBIGUE': 3}

In [22]:
from transformers import CamembertConfig, CamembertForTokenClassification

# Récupérer les noms des labels

num_labels = len(label_names)

# Définir la configuration du modèle
config = CamembertConfig(
    vocab_size=500_000,  # Taille du vocabulaire
    num_labels=num_labels,  # Nombre de labels dans votre dataset
    hidden_size=768,  # Taille des embeddings
    num_attention_heads=12,  #le lien entre les tokens, jusqu'au 12 eme token
    num_hidden_layers=12,  # Nombre de couches
)

config.label2id = label2id
config.id2label = id2label

# Initialiser le modèle avec la configuration
model = CamembertForTokenClassification(config)

In [23]:
config

CamembertConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "DET",
    "2": "APPO",
    "3": "AMBIGUE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "AMBIGUE": 3,
    "APPO": 2,
    "DET": 1,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 500000
}

In [ ]:
label_names

['O', 'DET', 'APPO', 'AMBIGUE']

In [24]:
from transformers import TrainingArguments

model_name = "relatives_psr"
model_name =  f"relatives_{task}"

args = TrainingArguments(
    model_name,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=5,
    #weight_decay=0.01,
    push_to_hub=False,

)

In [25]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [26]:
history = trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.060219,0.739755,0.630885,0.569180,0.976851
2,No log,0.058347,0.682092,0.630401,0.595208,0.978886
3,No log,0.058382,0.660397,0.609801,0.561016,0.977614
4,No log,0.056385,0.586838,0.610673,0.598057,0.976088
5,No log,0.053619,0.664043,0.645029,0.648263,0.980412


In [27]:
trainer.push_to_hub(commit_message="Training complete")

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

events.out.tfevents.1717543689.eefaff9f758d.4259.0:   0%|          | 0.00/7.57k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/djamina/relatives_psr/commit/27a1fb237f2b4a5594c5548f1407ef100baa5b05', commit_message='Training complete', commit_description='', oid='27a1fb237f2b4a5594c5548f1407ef100baa5b05', pr_url=None, pr_revision=None, pr_num=None)

## Inference